In [1]:
import numpy as np
import pandas as pd
import os  

In [140]:
# Cargamos los datsets
clientes = pd.read_csv('data\e-comerce_Olist_dataset\olist_customers_dataset.csv')
geolocalizacion = pd.read_csv('data\e-comerce_Olist_dataset\olist_geolocation_dataset_coregido.csv')


items = pd.read_csv('data\e-comerce_Olist_dataset\olist_order_items_dataset.csv', parse_dates=['shipping_limit_date'])
metodo_de_pago = pd.read_csv('data\e-comerce_Olist_dataset\olist_order_payments_dataset.csv')
reviews = pd.read_csv('data\e-comerce_Olist_dataset\olist_order_reviews_dataset.csv', parse_dates=['review_creation_date', 'review_answer_timestamp'])
ordenes = pd.read_csv('data\e-comerce_Olist_dataset\olist_orders_dataset.csv', parse_dates=['order_purchase_timestamp', 'order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date'])
productos = pd.read_csv('data\e-comerce_Olist_dataset\olist_products_dataset.csv')
vendedores = pd.read_csv('data\e-comerce_Olist_dataset\olist_sellers_dataset.csv')
productos_info = pd.read_csv('data\e-comerce_Olist_dataset\product_category_name_translation.csv')

marketing_ofertas_cerradas = pd.read_csv("data\Olist_Funnel_marketing\olist_closed_deals_dataset.csv", parse_dates=['won_date']) 
marketing_qualified_leads = pd.read_csv("data\Olist_Funnel_marketing\olist_marketing_qualified_leads_dataset.csv", parse_dates=['first_contact_date'])

In [ ]:
# Creamos las carpetas para guardar nuestros archivos etl y auxiliar que usaremos mas adelante
path_etl = "Dataset_etl"
path_aux = "Dataset_aux"
os.makedirs(path_etl, exist_ok=True)  
os.makedirs(path_aux, exist_ok=True)  


### ETL Geolocalizacion

In [4]:
geolocalizacion.head(3)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP


In [6]:
geolocalizacion.geolocation_zip_code_prefix.nunique()

19015

In [5]:
geoloc_filtrado = geolocalizacion.groupby('geolocation_zip_code_prefix').mean().reset_index()


In [7]:
geoloc_filtrado.geolocation_zip_code_prefix.nunique()

19015

In [11]:
geoloc_filtrado.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
0,1001,-23.550190,-46.634024
1,1002,-23.548146,-46.634979
2,1003,-23.548994,-46.635731
3,1004,-23.549799,-46.634757
4,1005,-23.549456,-46.636733


In [12]:
geoloc_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19015 entries, 0 to 19014
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   geolocation_zip_code_prefix  19015 non-null  int64  
 1   geolocation_lat              19015 non-null  float64
 2   geolocation_lng              19015 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 445.8 KB


In [13]:
# agregamos columna de Id igual a la de index
geoloc_filtrado['IdGeolocalizacion'] = geoloc_filtrado.index

In [24]:
geoloc_filtrado

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,IdGeolocalizacion
0,1001,-23.550190,-46.634024,0
1,1002,-23.548146,-46.634979,1
2,1003,-23.548994,-46.635731,2
3,1004,-23.549799,-46.634757,3
4,1005,-23.549456,-46.636733,4
...,...,...,...,...
19010,99960,-27.953722,-52.025511,19010
19011,99965,-28.183372,-52.039850,19011
19012,99970,-28.343766,-51.874689,19012
19013,99980,-28.389129,-51.843836,19013


In [28]:
# Agregamos Ciudad y Estado
geoloc_filtrado['Ciudad'] = geoloc_filtrado.apply(lambda r: geolocalizacion[geolocalizacion.geolocation_zip_code_prefix==r.geolocation_zip_code_prefix].geolocation_city.values[0], axis =1)
geoloc_filtrado['Estado'] = geoloc_filtrado.apply(lambda r: geolocalizacion[geolocalizacion.geolocation_zip_code_prefix==r.geolocation_zip_code_prefix].geolocation_state.values[0], axis =1)


In [36]:
geoloc_filtrado.head(2)

,Id_Geolocalizacion,zip_code_prefix,Latitud,Longitud,Ciudad,Estado
0,0,1001,-23.550190,-46.634024,sao paulo,SP
1,1,1002,-23.548146,-46.634979,sao paulo,SP


In [35]:
# renombramos y reordenamos las columnas
geoloc_filtrado.rename(columns={'geolocation_zip_code_prefix': 'zip_code_prefix', 
                                'geolocation_lat': 'Latitud',
                                'geolocation_lng': 'Longitud',
                                'IdGeolocalizacion':'Id_Geolocalizacion'}, inplace=True)
geoloc_filtrado = geoloc_filtrado.reindex(columns=['Id_Geolocalizacion', 'zip_code_prefix', 'Latitud', 
                                'Longitud','Ciudad','Estado'])

In [37]:
# En la carpeta Dataset_etl guardamos los archivos luego del proceso de ETL
geoloc_filtrado.to_csv("{}/geolocalizacion_etl.csv".format(path_etl), index=False)

### ETL a clientes

In [141]:
clientes.head(2)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP


In [142]:
clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [143]:
# no tenemos id repetidos
clientes.customer_id.nunique()

99441

In [144]:
# esta bien que hallan repetidos porque los unicos de este item son los clientes, pero estan guardados como compras unicas... 
# con el customer_id que refiere al id de la compra de cada cliente

clientes.customer_unique_id.nunique()

96096

Hacemos una tabla sola de clientes?? en principio les haremos el ETL para agregar el ID de geolocalizacion

Chequeamos que los zip-codes de la tabla clientes estan en la tabla geolocalizacion

In [145]:
geoloc_unique_code = geoloc_filtrado.zip_code_prefix.unique()
clientes_unique_code = clientes.customer_zip_code_prefix.unique()
zip_codes_dif = [item for item in clientes_unique_code if item not in geoloc_unique_code]
len(zip_codes_dif)

157

Hay 157 codigos postales de clientes que no coinciden con los codigos de la tabla geolocalizacion.
Para cada ciudad en la tabla de geolocalizacion hay mas de un zip-code... lo que vamos a hacer es cambiar los de los clientes por la mediana de los de geolocalizacion

In [146]:
geoloc_unique_code = geolocalizacion.geolocation_zip_code_prefix.unique()
clientes_unique_code = clientes.customer_zip_code_prefix.unique()
zip_codes_missing = []
for item in clientes_unique_code:
    if item not in geoloc_unique_code:
        try:
            city = clientes.loc[clientes['customer_zip_code_prefix'] == item].customer_city.values[0]
            zip_code = geolocalizacion[geolocalizacion.geolocation_city==city].geolocation_zip_code_prefix.mode()[0]
            clientes['customer_zip_code_prefix'] = clientes['customer_zip_code_prefix'].replace(item, zip_code)
        except:
            zip_codes_missing.append(item)

In [147]:
len(zip_codes_missing)

42

In [148]:
(len(zip_codes_missing)/clientes.shape[0])*100

0.04223609979787009

Todavia tenemos 42 zip-code que no se encuentran en geolocalizacion. Estos corresponden al 0.04% de los datos... probamos con Levenshtein a ver si podemos evaluar nombres mal escritos sino los guardamos en una tabla auxiliar

In [73]:
# ciudades a evaluar
ciudades_eval = []
for item in zip_codes_missing:
    ciudades_eval.append(clientes.loc[clientes['customer_zip_code_prefix'] == item].customer_city.values[0])
#ciudades de geolocalizacion
ciudades_geoloc = geoloc_filtrado.Ciudad.unique()

In [48]:
from hermetrics.levenshtein  import Levenshtein 

lev = Levenshtein ()

resultado_favorable_lev = {}
resultado_desfavorable_lev = {}

for x in ciudades_eval:
    puntuacion = 0
    i=-1
    for y in ciudades_geoloc:
        puntos = lev.similarity(x, y)
        i+=1
        if puntos > puntuacion:
            puntuacion = puntos
            indice = i
    if puntuacion>=0.7:
        resultado_favorable_lev[x] = [ciudades_geoloc[indice], puntuacion]
    else:
        resultado_desfavorable_lev[x] = [ciudades_geoloc[indice],puntuacion]

In [49]:
len(resultado_favorable_lev)

21

In [50]:
len(resultado_desfavorable_lev)

22

In [51]:
resultado_favorable_lev

{'glaura': ['planura', 0.7142857142857143],
 'nossa senhora do remedio': ['nossa senhora dos remedios',
  0.9230769230769231],
 'cipo-guacu': ['ipanguacu', 0.7],
 'sao francisco do humaita': ['sao francisco do gloria', 0.7916666666666666],
 'palmeirinha': ['palmeirina', 0.9090909090909091],
 'mampituba': ['marituba', 0.7777777777777778],
 'passagem': ['passabém', 0.75],
 'sao sebastiao da serra': ['sao sebastiao da barra', 0.9090909090909091],
 'doce grande': ['laje grande', 0.7272727272727273],
 'itabi': ['itobi', 0.8],
 'bora': ['borba', 0.8],
 'ibitioca': ['ibitiuva', 0.75],
 'jaguarembe': ['jaguaribe', 0.8],
 'jacuipe': ['jacuípe', 0.8571428571428572],
 'conceicao do formoso': ['conceicao dos ouros', 0.75],
 'poco de pedra': ['pocao de pedras', 0.8666666666666667],
 'jaua': ['maua', 0.75],
 'sao miguel do cambui': ['sao miguel dos campos', 0.8095238095238095],
 'palmital de minas': ['para de minas', 0.7058823529411764],
 'sao sebastiao do paraiba': ['sao sebastiao do paraiso', 0.91

Esta complicado arreglarlos.. asi que en principio guardamos en una tabla auxiliar las filas que no logramos corregir

In [150]:
zip_codes_missing[2]

65137

In [152]:
clientes[clientes.customer_zip_code_prefix==zip_codes_missing[2]].index.values

array([ 2321,  5420, 71065], dtype=int64)

In [149]:
indexes = []
for item in zip_codes_missing:
    ind = clientes[clientes.customer_zip_code_prefix==item].index.values
    for i in ind:
        indexes.append(i)

In [153]:
len(indexes)

49

In [154]:
cientes_aux = clientes.iloc[indexes]
cientes_aux.shape

(49, 5)

In [155]:
cientes_aux.head(2)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
1421,c55a17a7c31353c35d48550b3aebc06f,578d5e6d37a20bbf631183feedfe2dea,35408,glaura,MG
2166,7557541c9c578082c892cf185c3b0b47,7ee645e84a436c9dad3d5af253162fe7,8980,nossa senhora do remedio,SP


In [157]:
clientes.drop(index = indexes, axis=0, inplace=True)

In [158]:
clientes.shape

(99392, 5)

In [160]:
# chequeamos que los zip_codes coincidan
geoloc_code = geoloc_filtrado.zip_code_prefix.unique()
clientes_code = clientes.customer_zip_code_prefix.unique()
for item in clientes_code:
    if item not in geoloc_code:
        print(item)

[]

Ahora que eliminamos estos valores problema aplicamos el id_Geolocalizacion a la tabla clientes

In [64]:
geoloc_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19015 entries, 0 to 19014
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id_Geolocalizacion  19015 non-null  int64  
 1   zip_code_prefix     19015 non-null  int64  
 2   Latitud             19015 non-null  float64
 3   Longitud            19015 non-null  float64
 4   Ciudad              19015 non-null  object 
 5   Estado              19015 non-null  object 
dtypes: float64(2), int64(2), object(2)
memory usage: 891.5+ KB


In [161]:
clientes['Id_Geolocalizacion'] = clientes.apply(lambda r: geoloc_filtrado[geoloc_filtrado.zip_code_prefix==r.customer_zip_code_prefix].Id_Geolocalizacion.values[0], axis = 1)


In [162]:
clientes.head(3)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,Id_Geolocalizacion
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,5358
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,4279
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,84


In [163]:
clientes_etl_2 = clientes.drop(['customer_zip_code_prefix','customer_city','customer_state'],axis=1)

clientes_etl_2.head()

,customer_id,customer_unique_id,Id_Geolocalizacion
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,5358
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,4279
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,84
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,4028
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,4835


Guardo las dos tablas en un csv.. para evaluar despues cual usar..
Tambien guardamos la auxiliar

In [173]:
# guardamos clientes_aux en un csv
cientes_aux.to_csv("{}/cientes_aux.csv".format(path_aux), index=False)

# Guardamos las etl
clientes.to_csv("{}/clientes_etl_con_ciudadyestado.csv".format(path_etl), index=False)
clientes_etl_2.to_csv("{}/clientes_etl_solo_con_IdGeoloc.csv".format(path_etl), index=False)

### ETL Vendedores

In [165]:
vendedores.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [166]:
vendedores.shape

(3095, 4)

In [167]:
# no tenemos id repetidos :)
vendedores.seller_id.nunique()

3095

Chequeamos que los zip-codes de la tabla vendedores estan en la tabla geolocalizacion

In [168]:
geoloc_unique_code = geoloc_filtrado.zip_code_prefix.unique()
vendedores_unique_code = vendedores.seller_zip_code_prefix.unique()
zip_codes_dif = [item for item in vendedores_unique_code if item not in geoloc_unique_code]
len(zip_codes_dif)

7

Hay 7 codigos postales de clientes que no coinciden con los codigos de la tabla geolocalizacion.
Para cada ciudad en la tabla de geolocalizacion hay mas de un zip-code... lo que vamos a hacer es cambiar los de los clientes por la mediana de los de geolocalizacion

In [169]:
geoloc_unique_code = geolocalizacion.geolocation_zip_code_prefix.unique()
vendedores_unique_code = vendedores.seller_zip_code_prefix.unique()
zip_codes_missing = []
for item in vendedores_unique_code:
    if item not in geoloc_unique_code:
        try:
            city = vendedores.loc[vendedores['seller_zip_code_prefix'] == item].seller_city.values[0]
            zip_code = geolocalizacion[geolocalizacion.geolocation_city==city].geolocation_zip_code_prefix.mode()[0]
            vendedores['seller_zip_code_prefix'] = vendedores['seller_zip_code_prefix'].replace(item, zip_code)
        except:
            zip_codes_missing.append(item)
zip_codes_missing

[]

Logramos arreglar todos :)
Ahora aplicamos el id_Geolocalizacion a la tabla vendedores

In [170]:
vendedores['Id_Geolocalizacion'] = vendedores.apply(lambda r: geoloc_filtrado[geoloc_filtrado.zip_code_prefix==r.seller_zip_code_prefix].Id_Geolocalizacion.values[0], axis = 1)


In [171]:
vendedores

,seller_id,seller_zip_code_prefix,seller_city,seller_state,Id_Geolocalizacion
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,4810
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP,5209
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ,6354
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP,1833
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP,4772
...,...,...,...,...,...
3090,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR,17108
3091,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC,17326
3092,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP,2180
3093,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS,18514


In [172]:
vendedores_etl_2 = vendedores.drop(['seller_zip_code_prefix','seller_city','seller_state'],axis=1)

vendedores_etl_2.head()

,seller_id,Id_Geolocalizacion
0,3442f8959a84dea7ee197c632cb2df15,4810
1,d1b65fc7debc3361ea86b5f14c68d2e2,5209
2,ce3ad9de960102d0677a81f5d0bb7b2d,6354
3,c0f3eea2e14555b6faeea3dd58c1b1c3,1833
4,51a04a8a6bdcb23deccc82b0b80742cf,4772


Guardo las dos tablas en un csv.. para evaluar despues cual usar..

In [174]:

# Guardamos las etl
vendedores.to_csv("{}/vendedores_etl_con_ciudadyestado.csv".format(path_etl), index=False)
vendedores_etl_2.to_csv("{}/vendedores_etl_solo_con_IdGeoloc.csv".format(path_etl), index=False)